In [77]:
import pandas as pd
import gzip
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import Word2Vec

import re
import nltk
from nltk.corpus import stopwords

In [38]:
# Get the list of stopwords

nltk.download('stopwords')
stopwords_list = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Konstantinos
[nltk_data]     Razgkel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [95]:
df = pd.read_csv("data/training_data.tsv.gz", sep="\t", header=None).head(10000)
df.rename(columns={0: 'index', 1: 'title', 2: 'text', 3: 'labels'}, inplace=True)
df.drop('index', axis=1, inplace=True)

In [98]:
X = df.drop('labels', axis=1)
X = X.applymap(lambda x: re.sub(r'<.*?>|[^\w\s]', '', x.lower())).applymap(lambda x: ' '.join([word for word in x.split() if word not in stopwords_list]))
y = df['labels'].str.get_dummies(',')

X  = X["title"] + X["text"]

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(X)

# Split the data into training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Initialize the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the KNN classifier
knn.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluate the accuracy of the model
f1 = f1_score(y_test, y_pred, average='macro')
print("f1:", f1)


Accuracy: 0.384
f1: 0.5670309840442009


In [83]:
test_df = pd.DataFrame()

In [86]:
test_df["text"] = y
test_df[]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [97]:
split_index = int(len(df) * 0.8)
# Split the data into training and test sets
X = df[:split_index]
y = df[split_index:]

X = X.applymap(lambda x: re.sub(r'<.*?>|[^\w\s]', '', x.lower())).applymap(lambda x: ' '.join([word for word in x.split() if word not in stopwords_list]))
y = df['labels'].str.get_dummies(',')

X  = X["title"] + X["text"]


# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Initialize the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the KNN classifier
knn.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn.predict(X_test)

# Evaluate the accuracy of the model
f1 = f1_score(y_test, y_pred, average='macro')
print("f1:", f1)

0       percentage width child element absolutely posi...
1       determine users timezonestandard way web serve...
2       print html document web servicewant print html...
3       wordbreak dashgiven relatively simple css div ...
4       give web sites icon iphoneset icon appears iph...
                              ...                        
7995    custom menu context menu mouse right clickcurr...
7996    using javascript change url used page bookmark...
7997    control styling html unordered listthought sty...
7998    send additional data autocomplete optionsfield...
7999    aspnet enter key form submit javascriptform wa...
Length: 8000, dtype: object
      css  html  javascript  jquery
0       1     1           0       0
1       0     1           1       0
2       0     1           0       0
3       1     1           0       0
4       0     1           0       0
...   ...   ...         ...     ...
9995    0     1           0       0
9996    0     0           1       0
9997    0 

ValueError: Found input variables with inconsistent numbers of samples: [8000, 10000]

In [134]:
actual_data = y[len(y)- len(y_pred):]
actual_data.reset_index(drop=True)
predicted_data = pd.DataFrame(y_pred, columns=["css", "html", "javascript", "jquery"])

one_size = len(predicted_data)
seco_size = len(actual_data)

similar = sum([data_pred.equals(data_actual) for (_, data_pred), (_, data_actual) in zip(predicted_data.iterrows(), actual_data.iterrows())])


f1_score_new = 1/(index+1) * (2 * similar / (one_size + seco_size))

print("{:.8f}".format(f1_score_new))

0.00019200


In [128]:
(1/(index+1)) * similar /(one_size + seco_size)

9.6e-05